In [ ]:
# To be run early on results day.
# Will take about an hour to generate transcripts for all students.

In [ ]:
import os.path
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
import getpass
from weasyprint import HTML, CSS
from tqdm import tqdm

In [ ]:
loginurl = 'https://bluecastle-results.nottingham.ac.uk/login'
studenturl = 'https://bluecastle-results.nottingham.ac.uk/Administration/StudentView/StudentView'

In [ ]:
# this is a OneDrive folder for sharing with tutors
outpath = '/Users/spb/OneDrive/The University of Nottingham/Physics_Tutors - Documents/Transcripts/'

In [ ]:
style = CSS(string='''
table  {border-collapse: collapse;}
td, th {padding-left: 10px;}
''')

In [ ]:
password = getpass.getpass('\tpassword: ')

In [ ]:
loginpayload = {'UserName': 'ppzsb1',
                'Password': password}

In [ ]:
def transcript_from_response(response, tutor, year):
    soup = BeautifulSoup(response.content, features="html5lib")
    marks = soup.find('h2', text='My Marks').parent
    info = marks.p.contents
    name = info[10].strip() + ' ' + info[6].strip()
    marks.find('p', attrs={'class': 'transcript-print-view'}).decompose()
    marks.find('div', attrs={'class': 'disclaimer'}).decompose()
    marks.find('h2', text='My Marks').decompose()
    marks.find_all('p')[1].decompose()
    tag = soup.new_tag("h1")
    tag.string = name
    marks.insert(0, tag)
    tag = soup.new_tag("h2")
    tag.string = "Marks transcript - January 2020"
    marks.insert(1, tag)
    tag = soup.new_tag("h3")
    tag.string = f"Year {year}"
    marks.insert(2, tag)
    filename = f"{name.replace(' ', '_')}.pdf"
    tutor_year_path = os.path.join(outpath, tutor, f"year{year}")
    os.makedirs(tutor_year_path, exist_ok=True)
    filename = os.path.join(tutor_year_path, filename)
    HTML(string=str(marks)).write_pdf(filename, stylesheets=[style])

In [ ]:
fn = '/Users/spb/OneDrive/The University of Nottingham/Physics_Tutors - Documents/General/Tutor_List_070220.xlsx'
df = pd.read_excel(fn)
sids = df['Student Id']
sids = np.where((sids > 10000000) & (sids < 20000000), sids - 10000000, sids)
df['Student Id'] = sids.astype('str')
df = df.set_index('Student Id')
df['status'] = 'not tried'

In [ ]:
# For testing:
# df = df.sample(10)

In [ ]:
with requests.Session() as s:
    s.post(loginurl, data=loginpayload)
    iterator = tqdm(df.index)
    for sid in iterator:
        if df.loc[sid, 'status'] == 'not tried':
            tutor = df.loc[sid, 'Tutor']
            year = df.loc[sid, 'Year on Course ']
            studentpayload = {'selectedStudent': sid}
            r = s.post(studenturl, data=studentpayload)
            student = f"{sid} {df.loc[sid, 'First Name(s)']} {df.loc[sid, 'Surname']}"
            try:
                transcript_from_response(r, tutor, year)
                iterator.set_description(f'{student}: transcript created')
                df[sid, 'status'] = 'created'
            except:
                iterator.set_description(f'{student}: not found on BlueCastle')
                df[sid, 'status'] = 'not found'